In [2]:
import pandas as pd

pre_make_pivot_table_df = pd.read_excel("../data/pre_make_pivot_table.xlsx", dtype = {"gameId":"str"})

In [3]:
fromvalue_pivot = pd.pivot_table(
    pre_make_pivot_table_df,
    values = "formvalue",
    index = ["gameId"],
    columns = ["side", "role", "elements"]
).reset_index()
fromvalue_pivot.columns = ['_'.join(col) if isinstance(col, tuple) else col for col in fromvalue_pivot.columns]
fromvalue_pivot.rename(columns = {"gameId__" : "gameId"}, inplace = True)
teamwinrate_df = pre_make_pivot_table_df[["gameId", "side", "teamWinrate"]].drop_duplicates()

teamwinrate_df = teamwinrate_df.pivot(index='gameId', columns='side', values='teamWinrate')

# Subtract Red from Blue to get the new column
teamwinrate_df['teamWinrateDiff'] = teamwinrate_df['Blue'] - teamwinrate_df['Red']
teamwinrate_df.drop(["Blue", "Red"], axis=1, inplace=True)

# Reset index to turn gameId back into a column
teamwinrate_diff_df = teamwinrate_df.reset_index()
headtohead_and_winner_df = pre_make_pivot_table_df[["gameId", "headtoHeadWinrate", "winner"]].drop_duplicates()
final_pivot_table = fromvalue_pivot.merge(teamwinrate_diff_df, on = "gameId", how = "left")
final_pivot_table = final_pivot_table.merge(headtohead_and_winner_df, on = "gameId", how = "left")
#final_pivot_table.to_excel("../data/dataset.xlsx")
final_pivot_table.head()

,gameId,Blue_Adc_championDamageShare,Blue_Adc_creepScorePerTime,Blue_Adc_goldEarnedPerTime,Blue_Adc_kda,Blue_Adc_wardsScorePerTime,Blue_Jgl_championDamageShare,Blue_Jgl_creepScorePerTime,Blue_Jgl_goldEarnedPerTime,Blue_Jgl_kda,...,Red_Spt_kda,Red_Spt_wardsScorePerTime,Red_Top_championDamageShare,Red_Top_creepScorePerTime,Red_Top_goldEarnedPerTime,Red_Top_kda,Red_Top_wardsScorePerTime,teamWinrateDiff,headtoHeadWinrate,winner
0,106269654661533486,0.292294,0.163173,6.974498,5.200,0.016950,0.145844,0.093639,5.574723,6.056667,...,5.520000,0.049224,0.255484,0.145089,6.799953,6.573333,0.015360,-0.098128,0.250000,Red
1,106269654661533487,0.282222,0.157684,6.691084,3.950,0.016688,0.147539,0.089636,5.469117,4.906667,...,5.786667,0.049284,0.267624,0.145650,6.922345,7.793333,0.015350,-0.119048,0.200000,Red
2,106269654661599026,0.283189,0.147146,7.363543,7.735,0.019430,0.157957,0.097901,5.713389,8.435000,...,5.582619,0.045983,0.272596,0.151100,7.034842,5.393333,0.015188,-0.066667,0.200000,Red
3,106269654661599027,0.259536,0.179423,7.427517,6.000,0.018119,0.174771,0.099356,5.900707,7.677143,...,6.220000,0.047948,0.273684,0.145948,6.929040,8.130000,0.013809,-0.086957,0.166667,Red
4,106269654661599028,0.277617,0.148115,7.255165,8.540,0.020133,0.169911,0.104458,6.019701,8.910000,...,2.632619,0.047078,0.297359,0.149443,7.136887,5.601667,0.015457,0.106383,0.857143,Blue


In [4]:
final_pivot_table_removed = final_pivot_table[["gameId", "headtoHeadWinrate", "teamWinrateDiff",\
                                               "Blue_Top_kda", "Blue_Jgl_kda", "Blue_Mid_kda", "Blue_Adc_kda", "Blue_Spt_kda",\
                                                "Red_Top_kda", "Red_Jgl_kda", "Red_Mid_kda", "Red_Adc_kda", "Red_Spt_kda",\
                                                    "winner"]]
final_pivot_table_removed.head()

,gameId,headtoHeadWinrate,teamWinrateDiff,Blue_Top_kda,Blue_Jgl_kda,Blue_Mid_kda,Blue_Adc_kda,Blue_Spt_kda,Red_Top_kda,Red_Jgl_kda,Red_Mid_kda,Red_Adc_kda,Red_Spt_kda,winner
0,106269654661533486,0.250000,-0.098128,2.880000,6.056667,6.526667,5.200,4.541667,6.573333,6.625000,7.182857,6.251667,5.520000,Red
1,106269654661533487,0.200000,-0.119048,2.480000,4.906667,5.026667,3.950,3.341667,7.793333,7.485000,7.449524,7.318333,5.786667,Red
2,106269654661599026,0.200000,-0.066667,8.060000,8.435000,8.049524,7.735,6.170000,5.393333,3.937500,5.705193,5.566667,5.582619,Red
3,106269654661599027,0.166667,-0.086957,5.676667,7.677143,5.641667,6.000,4.882619,8.130000,8.660000,7.849524,7.885000,6.220000,Red
4,106269654661599028,0.857143,0.106383,7.630000,8.910000,6.409524,8.540,6.236667,5.601667,7.753333,5.741667,5.750000,2.632619,Blue


In [5]:
final_pivot_table_removed.to_excel("../data/dataset_draft2.xlsx")